In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import time, math
from sklearn.preprocessing import scale
from scipy import sparse
from sklearn.decomposition import NMF
from sklearn.exceptions import ConvergenceWarning
import _pickle as cPickle
import warnings
from matplotlib import pyplot as plt
import numpy as np
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)

In [15]:
def hitRatio(t1, t2):
    cnt = 0
    for i in t1:
        for j in t2:
            if i == j:
                cnt = cnt + 1
    return int(cnt/len(t1) *100)

In [16]:
def totalHitRatio(_nmf_rank, _persona):
    hit_per = []
    for p in _persona:
        hit = 0
        for i in range(len(p)):
            hit = hit + hitRatio(p[i], _nmf_rank[i])
        hit_per.append(round(hit / len(p),2))
    return hit_per

In [17]:
# NMF에 들어갈 Shape을 만들어 준다.
def GetShape(filename):
    df = pd.read_csv(filename, sep='|')
    n_users = len(df['persona'].unique())
    n_items = len(df['movie_id'].unique())
    return (n_users, n_items)

In [18]:
# R= (X,y), iin sparse format
def ConvertToDense(X, y, shape):
    row = X[:, 0]
    col = X[:, 1]
    data = y
    matrix_sparse = sparse.csr_matrix((data, (row, col)))
    R = matrix_sparse.todense()
    R = R[0:, 0:]
    R = np.asarray(R)
    return R

In [19]:
# 데이터를 로드한다.
def LoadData(filename, R_shape):
    df = pd.read_csv(filename, sep='|')
    X = df[['persona', 'movie_id', 'rating']].values
    y = df['rating'].values
    return X, y, ConvertToDense(X, y, R_shape)

In [20]:
def get_rmse(pred, actual):
    pred = pred[actual.nonzero()].flatten()     # Ignore nonzero terms
    actual = actual[actual.nonzero()].flatten() # Ignore nonzero terms
    return np.sqrt(mean_squared_error(pred, actual))

In [21]:
def make_recommendation_activeuser(item_info, R, prediction, user_idx, k=5):
    rated_items_df_user = pd.DataFrame(R).iloc[user_idx, :]                 # get the list of actual ratings of user_idx (seen movies)
    user_prediction_df_user = pd.DataFrame(prediction).iloc[user_idx,:]     # get the list of predicted ratings of user_idx (unseen movies)
    reco_df = pd.concat([rated_items_df_user, user_prediction_df_user, item_info], axis=1)   # merge both lists with the movie's title
    reco_df.columns = ['rating', 'prediction','title']
    #print('Preferred movies for user #', user_idx)
    #print(reco_df.sort_values(by='prediction', ascending=False)[:k])          # returns the 5 seen movies with the best actual ratings
    # print('Recommended movies for user #', user_idx)
    reco_df = reco_df[ reco_df['rating'] == 0]
    return reco_df.sort_values(by='prediction', ascending=False)[:k]
    # print (reco_df.sort_values(by='prediction', ascending=False)[:k])        # returns the 5 unseen movies with the best predicted ratings

In [22]:
def make_activeuser(item_info, R, prediction, user_idx, k=5):
    rated_items_df_user = pd.DataFrame(R).iloc[user_idx, :]                 # get the list of actual ratings of user_idx (seen movies)
    user_prediction_df_user = pd.DataFrame(prediction).iloc[user_idx,:]     # get the list of predicted ratings of user_idx (unseen movies)
    reco_df = pd.concat([rated_items_df_user, user_prediction_df_user, item_info], axis=1)   # merge both lists with the movie's title
    reco_df.columns = ['rating', 'prediction','title']
    #print('Preferred movies for user #', user_idx)
    #print(reco_df.sort_values(by='prediction', ascending=False)[:k])          # returns the 5 seen movies with the best actual ratings
    # print('Recommended movies for user #', user_idx)
    reco_df = reco_df[ reco_df['rating'] == 0]
    return reco_df.sort_values(by='prediction', ascending=False)[:k]
    # print (reco_df.sort_values(by='prediction', ascending=False)[:k])        # returns the 5 unseen movies with the best predicted ratings

In [30]:
_PERSONA_NUM_ = ['30']
_TOP_N_ = 12
_THRESHOLD_ = 3.5

In [37]:
print("Start!")
for i in _PERSONA_NUM_:   
    # Load the data set
    R_shape = GetShape('./files/persona_data.d')
    X, y, R = LoadData('./files/persona_data.d', R_shape)
    item_info = pd.read_csv('./files/u.item', sep='|', header=None, usecols=[1], engine='python')   # Information about the item
    item_info.columns = ['title']

    # Choose a model: NMF
    parametersNMF = {
        'n_components': 15,  # number of latent factors
        'init': 'random',
        'random_state': 0,
        'alpha': 0.001,  # regularization term
        'l1_ratio': 0,  # set regularization = L2
        'max_iter': 15
    }

    estimator = NMF(**parametersNMF)
    R_train = ConvertToDense(X, y, R_shape)
    R_test = ConvertToDense(X_test, y_test, R_shape)
    
    estimator.fit(R_train)
    Theta = estimator.transform(R_train)  # user features
    M = estimator.components_.T  # item features
    R_pred = M.dot(Theta.T)
    R_pred = R_pred.T
    
    print ('RMSE test:', get_rmse(R_pred, R_test))
    
    #nmf_rank = []
    #df = pd.read_csv('./files/mp.data', sep='|')
    #np.save('nmf_pred', R_pred)
    #np.save('nmf_r', R)

print("End!")

Start!
RMSE test: 8.318156136202264
End!


In [38]:
def precisionRecallAtK(pred, topN, th):
    # pred
    pred.sort_values(by='prediction', ascending=False)[:k]

In [39]:
# make_recommendation_activeuser(item_info, R, R_pred, user_idx=j, k=_TOP_N_)
# make_recommendation_activeuser(item_info, R, prediction, user_idx, k=5):

topN = 50
total_precision = 0
total_recall = 0
p_div_cnt = 0
r_div_cnt = 0
for j in range(22):
    user_idx = j
    vCnt = 0
    rated_items_df_user = pd.DataFrame(R).iloc[user_idx, :]                 # get the list of actual ratings of user_idx (seen movies)
    user_prediction_df_user = pd.DataFrame(R_pred).iloc[user_idx,:]     # get the list of predicted ratings of user_idx (unseen movies)
    reco_df = pd.concat([rated_items_df_user, user_prediction_df_user, item_info], axis=1)   # merge both lists with the movie's title
    reco_df.columns = ['rating', 'prediction','title']
    # precision At K
    to_data = reco_df.sort_values(by='prediction', ascending=False)[:topN]
    n_to_data = np.array(to_data)
    n_rel = 0
    for r in n_to_data:
        if (r[0]> 0):
            vCnt = vCnt + 1
            if( r[1] >= _THRESHOLD_ ):
                n_rel = n_rel + 1
        
    # precisionAtK = k / 
    if(vCnt != 0 ):
        p_div_cnt = p_div_cnt + 1
    
    total_precision = total_precision + ((n_rel+0.00000001) / (vCnt+0.00000001))

    # recall At K
    vCnt = 0
    reco_df.sort_values(by='rating', ascending=False)[:topN]
    n_to_data = np.array(to_data)
    n_rel = 0
    for r in n_to_data:
        if (r[0]> 0):
            vCnt = vCnt + 1
            if( r[0] >= _THRESHOLD_ ):
                n_rel = n_rel + 1
    
    if(vCnt != 0 ):
        r_div_cnt = r_div_cnt + 1
        
    total_recall = total_recall + ((n_rel+0.00000001) / (vCnt+0.00000001))
print((total_precision / p_div_cnt) * 100 )
print((total_recall / r_div_cnt) * 100)

93.1818181831818
93.54545454674546


In [40]:
import time
start = time.time()
make_recommendation_activeuser(item_info, R, R_pred, user_idx=57, k=1000)
print(time.time()-start)

IndexError: single positional indexer is out-of-bounds